In [3]:
import os
import json
import random
import string
import pandas as pd
import requests as req
import multiprocessing as mp
from tqdm import tqdm

# Preprocess PatentsView Data - extract all Cleantech Patents

In [5]:
df_patent = pd.read_csv('/mnt/hdd01/patentsview/g_patent.csv')

/tmp/ipykernel_1171359/1921840389.py:1: DtypeWarning: Columns (0,4) have mixed types. Specify dtype option on import or set low_memory=False.
  df_patent = pd.read_csv('/mnt/hdd01/patentsview/g_patent.csv')


In [6]:
# Convert patent_id to string
df_patent['patent_id'] = df_patent['patent_id'].astype(str)

In [7]:
# Read in cpc_current.tsv
df_cpc = pd.read_csv('/mnt/hdd01/patentsview/g_cpc_current.csv')

# Convert patent_id to string
df_cpc['patent_id'] = df_cpc['patent_id'].astype(str)

In [8]:
# Match df_patent and df_cpc on patent_id - inner join
df_patentsview_patent_cpc = pd.merge(df_patent, df_cpc, how='inner', on='patent_id', validate='many_to_many')

In [9]:
# Drop all patents without title or abstract
df_patentsview_patent_cpc = df_patentsview_patent_cpc.dropna(subset=['patent_title', 'patent_abstract'])

In [10]:
# Drop all rows where 'cpc_class' is unequal to 'Y02'
df_patentsview_patent_cpc = df_patentsview_patent_cpc[df_patentsview_patent_cpc['cpc_class'].str.contains('Y02')]

In [11]:
# Create list of patent_ids in PatentsView Cleantech dataset
df_patentsview_patent_cpc_id = df_patentsview_patent_cpc['patent_id'].drop_duplicates()
df_patentsview_patent_cpc_id.reset_index(drop=True, inplace=True)
df_patentsview_patent_cpc_id.to_csv('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/g_patent_ids_patentsview_cleantech.csv', index=False)

In [14]:
# Make new column 'cpc_subgroup' with characters until the first slash
df_patentsview_patent_cpc['cpc_subgroup'] = df_patentsview_patent_cpc['cpc_group'].str.split('/').str[0]

In [15]:
print(df_patentsview_patent_cpc.head())

    patent_id patent_type patent_date  \
145  10000011     utility  2018-06-19   
234  10000017     utility  2018-06-19   
253  10000017     utility  2018-06-19   
311  10000021     utility  2018-06-19   
347  10000025     utility  2018-06-19   

                                          patent_title  \
145  Supports for sintering additively manufactured...   
234  Method for mounting a vortex generator and mou...   
253  Method for mounting a vortex generator and mou...   
311  Method for manufacturing three-dimensional sha...   
347  Optimized cross-ply orientation in composite l...   

                                       patent_abstract wipo_kind  num_claims  \
145  To reduce distortion in an additively manufact...        B1          30   
234  The invention relates to a method for securing...        B2          21   
253  The invention relates to a method for securing...        B2          21   
311  There is provided a method for manufacturing a...        B2           4   
347 

In [17]:
# Group by 'patent_id' and aggregate the other columns into lists - THINK ABOUT THIS CODE - TOO HEFTY COMPUTING - DUPLICATION CHECK NEED TO BE DONE ELSEWHERE
df_patentsview_patent_cpc = df_patentsview_patent_cpc.groupby('patent_id').agg({
    'patent_type': 'first',
    'patent_date': 'first',
    'patent_title': 'first',
    'patent_abstract': 'first',
    'wipo_kind': 'first',
    'num_claims': 'first',
    'withdrawn': 'first',
    'filename': 'first',
    'cpc_sequence': list,
    'cpc_section': list,
    'cpc_class': list,
    'cpc_subclass': list,
    'cpc_group': list,
    'cpc_type': list,
    'cpc_symbol_position': list,
    'cpc_subgroup': list,
}).reset_index()

In [18]:
print(df_patentsview_patent_cpc.head())

  patent_id patent_type patent_date  \
0  10000011     utility  2018-06-19   
1  10000017     utility  2018-06-19   
2  10000021     utility  2018-06-19   
3  10000025     utility  2018-06-19   
4  10000033     utility  2018-06-19   

                                        patent_title  \
0  Supports for sintering additively manufactured...   
1  Method for mounting a vortex generator and mou...   
2  Method for manufacturing three-dimensional sha...   
3  Optimized cross-ply orientation in composite l...   
4  Washable, waterproof, sealable and reusable st...   

                                     patent_abstract wipo_kind  num_claims  \
0  To reduce distortion in an additively manufact...        B1          30   
1  The invention relates to a method for securing...        B2          21   
2  There is provided a method for manufacturing a...        B2           4   
3  A composite laminate has a primary axis of loa...        B2           9   
4  Disclosed are embodiments of a vers

In [19]:
# Restructure dataframe to nested dictionaries
df_patentsview_patent_cpc['cpc'] = df_patentsview_patent_cpc.apply(lambda row: {i+1: {'cpc_sequence': cpc_sequence, 'cpc_section': cpc_section, 'cpc_class': cpc_class, 'cpc_subclass': cpc_subclass, 'cpc_group': cpc_group, 'cpc_type': cpc_type, 'cpc_symbol_position': cpc_symbol_position, 'cpc_subgroup': cpc_subgroup} for i, (cpc_sequence, cpc_section, cpc_class, cpc_subclass, cpc_group, cpc_type, cpc_symbol_position, cpc_subgroup) in enumerate(zip(row['cpc_sequence'], row['cpc_section'], row['cpc_class'], row['cpc_subclass'], row['cpc_group'], row['cpc_type'], row['cpc_symbol_position'], row['cpc_subgroup']))}, axis=1)

In [20]:
# Drop original columns
df_patentsview_patent_cpc = df_patentsview_patent_cpc.drop(columns=['cpc_sequence', 'cpc_section', 'cpc_class', 'cpc_subclass', 'cpc_group', 'cpc_type', 'cpc_symbol_position', 'cpc_subgroup'])

In [23]:
# Save df_rel_on_science_patent_cpc_grouped to json
df_patentsview_patent_cpc.to_json('/mnt/hdd01/patentsview/Patentsview - Cleantech Patents/df_patentsview_patent_cpc_grouped_cleantech.json', orient='records', indent=2)

df_patentsview_patent_cpc = df_patentsview_patent_cpc.head(1000)
df_patentsview_patent_cpc.to_json('/home/thiesen/Documents/Cleantech_Concepts/df_patentsview_patent_cpc_grouped_cleantech.json', orient='records', indent=2)